In [1]:
import sqlite3 as sq
import pandas as pd

In [2]:
con = sq.connect("NPOPlayer.db")
def run_query(con, table, query):
    cur = con.cursor()
    cur.execute(query)
    results = cur.fetchall()
    return results

def convert_db_table_to_DF(con, table):
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"
    cols_init = run_query(con, table, header_query)
    cols = [cols_init[i][0] for i in range(len(cols_init))]
    content_query = "SELECT * from " + table
    data = run_query(con, table, content_query)
    df = pd.DataFrame(data, columns= cols)
    return df

In [4]:
df = convert_db_table_to_DF(con, "NPOPLAYER")

In [20]:
import spacy
from tqdm import tqdm

# use displacy to visually show the entities. 
from spacy import displacy

# load spacy model. Alternatively you can use en_core_web_lg
nlp = spacy.load("nl_core_news_sm")

In [32]:
# df_sub = df.copy()
df_sub = df.sample(6000).copy()

In [33]:
def process(x):
  # there are some pesky NaN in the data. Easy but not so elegant way to fix this.
  if pd.isna(x) == False:
    doc = nlp(x)
  else:
    doc = ''
  return doc


tqdm.pandas()
df_sub['plot_entities'] = df_sub['longSummary'].progress_apply(lambda x: process(x))


100%|██████████| 6000/6000 [00:52<00:00, 114.66it/s]


In [64]:
def get_events(x):
  events = []
  if x != '':
    for entities in x.ents:
      if entities.label_ == 'EVENT':
        events.append(entities.text)
      elif entities.label_ == 'LAW':
        events.append(entities.text)
      elif entities.label_ == 'LOC':
        events.append(entities.text)
      elif entities.label_ == 'PERSON':
        events.append(entities.text)
      elif entities.label_ == 'PRODUCT':
        events.append(entities.text)
      elif entities.label_ == 'WORK_OF_ART':
        events.append(entities.text)
  return events

# code goes here
df_sub['tags'] = df_sub['plot_entities'].progress_apply(lambda x: get_events(x))
df_sub['tags']

100%|██████████| 6000/6000 [00:00<00:00, 73365.04it/s]
